In [2]:
import investpy as ip
import yahooquery as yq
from pandas_datareader import data as pdr
import pandas as pd
from tqdm import trange
import yfinance as yf


In [3]:
df = pd.read_excel('./datas/MSCIeurope.xlsx')
df.head()

,Unnamed: 0,Ticker,Ticker Yahoo,Nom,Secteur,Equity class,Market Share,Weights,Valeur notionnelle,Actions,ISIN,Currency,Environment Score,Social Score,Governance Score,Total Score
0,0,NESN,NSRGY,NESTLE SA,Biens de consommation de base,Actions,"EUR 48 274 991,14",4.67,48274991.14,420838,CH0038863350,CHF,NaN,NaN,NaN,NaN
1,1,ASML,ASML,ASML HOLDING NV,Technologie de l'informatioon,Actions,"EUR 34 906 428,20",3.38,34906428.20,60802,NL0010273215,EUR,NaN,NaN,NaN,NaN
2,2,ROG,ROG.SW,ROCHE HOLDING PAR AG,Santé,Actions,"EUR 33 188 774,68",3.21,33188774.68,104989,CH0012032048,CHF,2.48,11.54,9.52,23.53
3,3,MC,MC.PA,LVMH,Biens de consommation cycliques,Actions,"EUR 29 028 191,40",2.81,29028191.40,41457,FR0000121014,EUR,0.76,5.93,5.67,12.37
4,4,NOVOB,NVO,NOVO NORDISK CLASS B,Santé,Actions,"EUR 28 242 417,87",2.73,28242417.87,247375,DK0060534915,DKK,NaN,NaN,NaN,NaN


### get yahoo ticker

In [2]:
def find_ticker(isin):
    try:
        company_name = ip.stocks.search_stocks(by='isin', value=str(isin))
        company_name = company_name["name"][0].split(' ')[0]
        symbol = yq.search(company_name)["quotes"][0]["symbol"]
    except:
        symbol = None
    return symbol

In [ ]:
for i in range(len(df)):
    df.loc[i, "Ticker Yahoo"] = find_ticker(df.loc[i, "ISIN"])
df = df[['Ticker', 'Ticker Yahoo', 'Nom', 'Secteur', 'Equity class', 'Market Share', 'Weights',
       'Valeur notionnelle', 'Actions', 'ISIN', 'Currency', ]] 

In [10]:
# total weight
print(df["Weights"].sum())
# lost weights
print(df.loc[(df['Ticker Yahoo'].isna()), 'Weights'].sum())

99.98999999999998
6.82


### get ESG scores

In [11]:
esg_scores = pd.DataFrame(columns = ['Ticker Yahoo', 'Environment Score', 'Social Score', 'Governance Score', 'Total Score'], index = range(len(df)))
# trange = tqdm(range(len(df)))

for i in trange(len(df)):
    ticker = df.loc[i, 'Ticker Yahoo']
    try:
        sus = yf.Ticker(ticker).sustainability
        scores = sus.loc[['environmentScore','socialScore','governanceScore','totalEsg'],'Value']
        esg_scores.loc[i] = [ticker, scores[0], scores[1], scores[2], scores[3]]
    except:
        pass
esg_scores


100%|██████████| 357/357 [09:28<00:00,  1.59s/it]


,Ticker Yahoo,Environment Score,Social Score,Governance Score,Total Score
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,ROG.SW,2.48,11.54,9.52,23.53
3,MC.PA,0.76,5.93,5.67,12.37
4,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
352,NaN,NaN,NaN,NaN,NaN
353,NaN,NaN,NaN,NaN,NaN
354,NaN,NaN,NaN,NaN,NaN
355,NaN,NaN,NaN,NaN,NaN


In [15]:
# count missing esg scores
print(esg_scores.isna().sum())

Ticker Yahoo         175
Environment Score    175
Social Score         175
Governance Score     175
Total Score          175
dtype: int64


In [12]:
# concat the two dataframes
df = pd.concat([df, esg_scores], axis=1)

In [14]:
# save the dataframe
df.to_excel('./datas/MSCIeurope.xlsx')

### create the datasets with the price for all ticker

In [4]:
prices = None
# trange = tqdm(range(len(df)))
tickers = yf.Tickers(df.loc[(df['Ticker Yahoo'].notna()), 'Ticker Yahoo'].tolist())

datas = tickers.history(period="max")
datas





[*********************100%***********************]  288 of 288 completed

2 Failed downloads:
- CNH=X: 1d data not available for startTime=-2208994789 and endTime=1669898091. Only 100 years worth of day granularity data are allowed to be fetched per request.
- SEK=F: 1d data not available for startTime=-2208994789 and endTime=1669898106. Only 100 years worth of day granularity data are allowed to be fetched per request.


Adj Close            Close                                 \
               CNH=X SEK=F    0QKY.IL    1COV.DE     6S=F        ABB   
Date                                                                   
1962-01-02       NaN   NaN        NaN        NaN      NaN        NaN   
1962-01-03       NaN   NaN        NaN        NaN      NaN        NaN   
1962-01-04       NaN   NaN        NaN        NaN      NaN        NaN   
1962-01-05       NaN   NaN        NaN        NaN      NaN        NaN   
1962-01-08       NaN   NaN        NaN        NaN      NaN        NaN   
...              ...   ...        ...        ...      ...        ...   
2022-11-25       NaN   NaN  48.959999  37.419998  1.06040  31.549999   
2022-11-28       NaN   NaN  49.049999  37.029999  1.05565  30.920000   
2022-11-29       NaN   NaN  48.869999  38.230000  1.05065  30.870001   
2022-11-30       NaN   NaN        NaN        NaN  1.06025  31.490000   
2022-12-01       NaN   NaN  49.110001  37.580002  1.06105        NaN   

                                                     ...     Volume        \
                  ABNB       AC.MX      AC.PA  ACST  ...        VTI WFDRX   
Date                                                 ...                    
1962-01-02         NaN         NaN        NaN   NaN  ...        NaN   NaN   
1962-01-03         NaN         NaN        NaN   NaN  ...        NaN   NaN   
1962-01-04         NaN         NaN        NaN   NaN  ...        NaN   NaN   
1962-01-05         NaN         NaN        NaN   NaN  ...        NaN   NaN   
1962-01-08         NaN         NaN        NaN   NaN  ...        NaN   NaN   
...                ...         ...        ...   ...  ...        ...   ...   
2022-11-25   97.669998  162.000000  24.790001  0.42  ...  1289100.0   0.0   
2022-11-28   95.300003  161.190002  24.709999  0.41  ...  3365800.0   0.0   
2022-11-29   95.379997  161.190002  24.879999  0.41  ...  2640700.0   0.0   
2022-11-30  102.139999  161.020004        NaN  0.42  ...  4198100.0   0.0   
2022-12-01         NaN         NaN  25.320000   NaN  ...        NaN   NaN   

                                                                        \
              WKL.AS    WLN.PA         WMB  WRTBF    YAR.OL     ZAL.DE   
Date                                                                     
1962-01-02       NaN       NaN         NaN    NaN       NaN        NaN   
1962-01-03       NaN       NaN         NaN    NaN       NaN        NaN   
1962-01-04       NaN       NaN         NaN    NaN       NaN        NaN   
1962-01-05       NaN       NaN         NaN    NaN       NaN        NaN   
1962-01-08       NaN       NaN         NaN    NaN       NaN        NaN   
...              ...       ...         ...    ...       ...        ...   
2022-11-25  364711.0  417307.0   2108100.0    0.0  188949.0   698451.0   
2022-11-28  527101.0  618545.0   5807500.0    0.0  361872.0   902156.0   
2022-11-29  540892.0  512498.0   6392000.0  110.0  260482.0  1549614.0   
2022-11-30       NaN       NaN  12456200.0    NaN       NaN        NaN   
2022-12-01  238564.0  219447.0         NaN    NaN  149315.0   824027.0   

                                   
             ZURN.SW        ^N100  
Date                               
1962-01-02       NaN          NaN  
1962-01-03       NaN          NaN  
1962-01-04       NaN          NaN  
1962-01-05       NaN          NaN  
1962-01-08       NaN          NaN  
...              ...          ...  
2022-11-25  255932.0  137274000.0  
2022-11-28  242314.0  170678500.0  
2022-11-29  222374.0  171833900.0  
2022-11-30       NaN  367098200.0  
2022-12-01   80629.0          0.0  

[15612 rows x 2014 columns]

In [8]:
datas['Close'].to_excel('./datas/hist_prices.xlsx')